The goal here is to create a "raster plot" that shows the reproducibility of a spike train to different repetitions of different stimulus: a DC current versus noise. 

In particular, we will attempt to replicate Figure 1 of [Mainen & Sejnowski (1995)](http://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.299.8560&rep=rep1&type=pdf) and use computational Neuroscience as a tool to better understanfd neurophysiological observations. 


# Mainen & Sejnowski, 1995

## context

The goal of this first task is to create a "raster plot" that shows the reproducibility of a spike train with repetitions of the same stimulus, as in this work in the [rodent retina](https://laurentperrinet.github.io/2019-04-03_a_course_on_vision_and_modelization/#/1/3) or in the [cat cortex (V1)](https://laurentperrinet.github.io/2019-04-03_a_course_on_vision_and_modelization/#/1/6).

## figure 1

Here, we will attempt to replicate Figure 1 of [Mainen & Sejnowski (1995)](http://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.299.8560&rep=rep1&type=pdf):

![Mainen Sejnowski 1995](http://i.stack.imgur.com/ixnrz.png "figure 1")


QUESTION: write a quick summary of the paper (max 5 lines) and why this result is *a priori* surprising.

# representation of time

## a gentle introduction to python and numpy

In [ ]:
dt = .5 # size of the time discretization step

In [ ]:
%whos

In [ ]:
import numpy as np
# help(np)

In [ ]:
np.lookfor('binary representation') 

In [ ]:
np.lookfor('evenly spaced values')

In [ ]:
time = np.linspace(-100, 1000, int(1100/dt))
# time = np.arange(-100, 1000, dt) # alternative


In [ ]:
time

In [ ]:
time.shape

In [ ]:
time[:10]

In [ ]:
time[0], time[1], time[-2], time[-1]

Creation of a DC current (version one):

In [ ]:
start = 0
end = 900
value = 142

def Inp(time=time, start=start, end=end, value=value):
    x=[]
    for t in range(len(time)):
        if start < time[t] < end :
            x.append(value)
        else:
            x.append(0)
    return x

I = Inp(time)

In [ ]:
%%timeit
I = Inp(time)

Vectorizing the code (version two):

In [ ]:
def Inp(time=time, start=start, end=end, value=value):
    I = np.zeros_like(time)
    I[time>start] = value
    I[time>end] = 0
    return I
  
I = Inp(time)

In [ ]:
%%timeit
I = Inp(time)

QUESTION: try to describe why the computation time to create the vector is different in the two versions

## a gentle introduction to matplotlib / pylab / pyplot

Matplotlib is the major library for plotting in the python ecosystem. Import it like this

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
fig_width = 15 # defining the size of plots
phi = (np.sqrt(5)+1)/2 # setting up the golden ratio
phi = phi**2

In [ ]:
np.lookfor('bar plot', module='matplotlib.pyplot')

In [ ]:
fig, ax = plt.subplots(figsize=(fig_width, fig_width/phi))
ax.plot(time, I)
ax.set_xlabel('Time (ms)')
ax.set_ylabel('I (pA)');

QUESTION: re-run this calculation by adjusting the parameters to match figure 1

If the code above looks scary, do not worry - Pythons are loving animals:

* a nice [one-day course in python](https://github.com/NeuromatchAcademy/precourse)
* resources abound  and [an army of geeks are ready to answer to any of your questions](https://stackoverflow.com/questions/tagged/python)

## a simple model of an integrate-and-fire neuron `leaky_IF`

Let's start with this membrane potential equation:

$$
\tau \cdot \frac{dV}{dt} = -(V - V_{rest}) + R*I(t)
$$

with emission of a spike if $V > V_{rest}$, and then $V= V_{rest}$ for $3 ms$.

In [ ]:
Vthreshold = -53
Vreset = -80
VRest = -70
Vspike = 30
R = 0.42
tau = 30

In [ ]:
def leaky_IF(time=time, inp=I, tau=tau, v0=VRest, R=R, 
             Vthreshold=Vthreshold, Vreset=Vreset, Vspike=Vspike, 
             VRest=VRest):
    V = np.ones_like(time)*v0
    dt = time[1] - time[0]
    for t in range(len(time)-1):
        dV = dt * (-(V[t] - VRest) + R*inp[t])/tau
        V[t+1] = V[t] + dV
        
        if V[t]>Vthreshold:
            V[t+1] = Vspike
        if V[t] == Vspike:
            V[t+1] = Vreset
         
    return V

QUESTION: Set the parameter $R$ to obtain about 10 spikes - what is the interpretation of this parameter and what is the unit of measurement?

In [ ]:
V = leaky_IF(time, I)

fig, ax = plt.subplots(1, 1, figsize=(fig_width, fig_width/phi))
ax.plot(time, V)
ax.axhline(Vthreshold, c='g', ls='--')
ax.set_xlabel('Time (ms)')
ax.set_ylabel('Membrane potential (mV)');

Focus: The linear part of the response

In [ ]:
value = 140
V = leaky_IF(time, Inp(time=time, start=100, end=500, value=value))

fig, ax = plt.subplots(1, 1, figsize=(fig_width, fig_width/phi))
ax.plot(time, V)
ax.axhline(Vthreshold, c='g', ls='--')
ax.axvline(100, c='k', ls='--')
ax.axvline(500, c='k', ls='--')
ax.axhline(VRest, c='r', ls='--')
ax.axline((100, VRest), slope=R*value/tau,  c='r', ls='--')
ax.set_xlabel('Time (ms)')
ax.set_xlim(80, 200)
ax.set_ylim(-70, -50)

ax.set_ylabel('Membrane potential (mV)');

QUESTION: What is the effect of $I_0$ on the discharge frequency?

In [ ]:
for rho in np.geomspace(0.5, 2., 5):
    I_0_ = rho*150
    print('I_0=', I_0_)
    V= leaky_IF(time, Inp(value=I_0_))

    fig, ax = plt.subplots(1, 1, figsize=(fig_width, fig_width/phi))
    ax.plot(time, V)
    ax.axhline(Vthreshold, c='g', ls='--')
    ax.set_ylim(-83, 40)
    ax.set_ylabel("Membrane potential (mV)")
    ax.set_xlabel('Time (ms)')
    plt.show()

Several tests show that this is perfectly reproducible, unlike figure 1A:

In [ ]:
n_trials = 5
V1 = np.zeros((n_trials,len(time)))

for i in range(n_trials):
    V1[i, :] = leaky_IF()

fig, ax = plt.subplots(1, 1, figsize=(fig_width, fig_width/phi))
ax.plot(time, V1.T)
ax.axhline(Vthreshold, c='g', ls='--')
ax.set_xlabel('Time (ms)')
ax.set_ylabel('Membrane potential (mV)');

QUESTION: this model seems not to reproduce the results, any explanation?

# Creating a noisy input

A linear scattering model allows you to simply create a noise:

In [ ]:
def genNoise(time=time, tau_n=10, I_n=100, I_0=30, start=start, end=end):
    dt = time[1] - time[0]
    x=np.ones_like(time)
    for t in range(len(x)-1):
        n = np.random.randn()*I_n
        x[t+1] = (1-dt/tau_n)*x[t] + (dt*n/tau_n)
    
    x += I_0
    x[time<start], x[time>end] = 0, 0
    
    return x

fig, ax = plt.subplots(1, 1, figsize=(fig_width, fig_width/phi))
ax.plot(time, genNoise())
ax.set_xlabel('Time (ms)')
ax.set_ylabel('I_b (pA)');

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(fig_width, fig_width/phi))
ax.plot(time, genNoise())
ax.set_xlabel('Time (ms)')
ax.set_ylabel('I_b (pA)');

QUESTION: does this model represent well the one in the paper? adjust $I_n$ and $I_0$ to get something that fits better.

## LIF neuron with noisy input

Let's now observe the response of our LIF neuron to this input:

In [ ]:
n_trials = 5
V1 = np.zeros((n_trials,len(time)))

for i in range(n_trials):
    V1[i, :] = leaky_IF(time, genNoise())

fig, ax = plt.subplots(1, 1, figsize=(fig_width, fig_width/phi))
ax.plot(time, V1.T)
ax.axhline(Vthreshold, c='g', ls='--')
ax.set_xlabel('Time (ms)')
ax.set_ylabel('Membrane potential (mV)');

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(fig_width, fig_width/phi))
ax.eventplot([dt*np.where(V1.T[:, i] == Vspike)[0] for i in range(0, n_trials)], 
              colors='black', lineoffsets=1, linelengths=0.9);
ax.set_ylabel('Trial number')
ax.set_xlabel('Time (ms)')
ax.set_xlim(time.min(), time.max())
ax.set_ylim(-.5, n_trials-.5);

QUESTION: adjust $I_n$ and $I_0$ to get something that better matches the observed output:

In [ ]:
for rho in np.geomspace(0.5, 2., 5):
    I_0_ = rho*250
    print('I_0=', I_0_)
    V= leaky_IF(time, genNoise(time, I_n=1000, I_0=I_0_))

    fig, ax = plt.subplots(1, 1, figsize=(fig_width, fig_width/phi))
    ax.plot(time, V)
    ax.set_ylim(-83, 40)
    ax.set_ylabel("Membrane potential (mV)")
    ax.set_xlabel('Time (ms)')
    plt.show()

In [ ]:
for rho in np.geomspace(0.5, 2., 5):
    I_n_ = rho*500
    print('I_n=', I_n_)
    V= leaky_IF(time, genNoise(time, I_n=I_n_, I_0=150))

    fig, ax = plt.subplots(1, 1, figsize=(fig_width, fig_width/phi))
    ax.plot(time, V)
    ax.set_ylim(-83, 40)
    ax.set_ylabel("Membrane potential (mV)")
    ax.set_xlabel('Time (ms)')

    plt.show()

QUESTION: Do we obtain something reproducible?

In [ ]:
n_trials = 5
V1 = np.zeros((n_trials,len(time)))

for i in range(n_trials):
    V1[i, :] = leaky_IF(time, genNoise(time, I_n=500, I_0=150))

fig, ax = plt.subplots(1, 1, figsize=(fig_width, fig_width/phi))
ax.plot(time, V1.T)
ax.axhline(Vthreshold, c='g', ls='--')
ax.set_xlabel('Time (ms)')
ax.set_ylabel('Membrane potential (mV)');

## "Frozen" noise?

QUESTION: what is the nature of the noise used in the article? why can it be described as [frozen noise] (https://www.oxfordreference.com/view/10.1093/oi/authority.20110803095836900)?

QUESTION: how to implement such a noise? what do you know about noise generators used in a computer?

In [ ]:
help(np.random.seed)

In [ ]:
np.random.randn()

In [ ]:
np.random.seed(42) # douglas adams
np.random.randn()

In [ ]:
np.random.randn()

In [ ]:
def genNoise(time=time, tau_n=10, I_n=10, I_0=15, seed=42, start=start, end=end):
    np.random.seed(seed)
    dt = time[1] - time[0]
    x=np.ones_like(time)
    for t in range(len(x)-1):
        n = np.random.randn()*I_n
        x[t+1] = (1-dt/tau_n)*x[t]+ (dt*n/tau_n)
    
    x += I_0
    x[time<start], x[time>end] = 0,0
    
    return x

fig, ax = plt.subplots(1, 1, figsize=(fig_width, fig_width/phi))
ax.plot(time, genNoise())
ax.set_xlabel('Time (ms)')
ax.set_ylabel('I_b (pA)');

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(fig_width, fig_width/phi))
ax.plot(time, genNoise())
ax.set_xlabel('Time (ms)')
ax.set_ylabel('I_b (pA)');

## Multiple trials
Here we show the conservation of the time of the spikes using a noisy input (frozen noise)

QUESTION: adjust the parameter $I_0$ and $I_n$ to obtain about ten action potentials:

In [ ]:
n_trials = 25
V1 = np.zeros((n_trials,len(time)))

for i in range(n_trials):
    V1[i, :] = leaky_IF(time, genNoise(I_n=800, I_0=15))

print('number of spikes per trial :', (V1>0).sum(axis=1))

fig, ax = plt.subplots(1, 1, figsize=(fig_width, fig_width/phi))
ax.plot(time, V1.T)
ax.set_xlabel('Time (ms)')
ax.set_ylabel('Membrane potential (mV)');

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(fig_width, fig_width/phi))
ax.eventplot([dt*np.where(V1.T[:, i] == Vspike)[0] for i in range(0, n_trials)], 
              colors='black', lineoffsets=1, linelengths=0.9);
ax.set_ylabel('Trial number')
ax.set_xlabel('Time (ms)')
ax.set_xlim(time.min(), time.max())
ax.set_ylim(-.5, n_trials-.5);

We reproduce panel B: with a frozen noise, the neuron traces are reproducible.

This also proves that we "forgot" to include a noise intrinsic to the dynamics of the neuron:

In [ ]:
def leaky_IF(time=time, inp=I, tau=30, v0=-65, R=R, 
                Vthreshold=Vthreshold, Vreset=Vreset, Vspike=Vspike, 
                VRest=VRest, b=40, seed=None):
    np.random.seed(seed)
    V = np.ones_like(time)*v0
    dt = time[1] - time[0]
    for t in range(len(time)-1):
        n=np.random.randn()
        dV = dt * (-(V[t] - VRest) + R*(inp[t]+b*n))/tau
        V[t+1] = V[t] + dV
        
        if V[t]>Vthreshold:
            V[t+1]= Vspike
        if V[t] == Vspike:
            V[t+1]=Vreset
         
    return V


Several tests show that with a square wave the spike times lose their reproducibility, as shown in the figure:

QUESTION: adjust $I_0$ and $I_n$ to obtain a qualitatively similar number of spikes at the neuron output. To do this, try to control the number of spikes:

In [ ]:
for rho in np.linspace(0.5, 2., 5):
    b_ = rho*100
    print('b =', b_)
    VA = np.zeros((n_trials,len(time)))

    for i in range(n_trials):
        VA[i, :] = leaky_IF(time, genNoise(I_n=100, I_0=150), b=b_)

    print('number of spikes per trial :', (VA>0).sum(axis=1))

    fig, ax = plt.subplots(1, 1, figsize=(fig_width, fig_width/phi))
    ax.eventplot([dt*np.where(VA.T[:, i] == Vspike)[0] for i in range(0, n_trials)], 
                colors='black', lineoffsets=1, linelengths=0.9);
    ax.set_ylabel('Trial number')
    ax.set_xlabel('Time (ms)')
    ax.set_xlim(time.min(), time.max())
    ax.set_ylim(-.5, n_trials-.5);
    plt.show()

In [ ]:
for rho in np.linspace(0.7, 1.3, 5):
    I_n_ = rho*200
    print('I_n =', I_n_)
    VA = np.zeros((n_trials,len(time)))

    for i in range(n_trials):
        VA[i, :] = leaky_IF(time, genNoise(I_n=I_n_, I_0=175))

    print('number of spikes per trial :', (VA>0).sum(axis=1))

    fig, ax = plt.subplots(1, 1, figsize=(fig_width, fig_width/phi))
    ax.eventplot([dt*np.where(VA.T[:, i] == Vspike)[0] for i in range(0, n_trials)], 
                colors='black', lineoffsets=1, linelengths=0.9);
    ax.set_ylabel('Trial number')
    ax.set_xlabel('Time (ms)')
    ax.set_xlim(time.min(), time.max())
    ax.set_ylim(-.5, n_trials-.5);
    plt.show()

QUESTION: see the influence of $I_0$ on the behavior

In [ ]:
for rho in np.linspace(0.7, 1.3, 5):
    print('rho=', rho)
    VA = np.zeros((n_trials,len(time)))

    for i in range(n_trials):
        VA[i, :] = leaky_IF(time, genNoise(I_n=500, I_0=rho*150))

    fig, ax = plt.subplots(1, 1, figsize=(fig_width, fig_width/phi))
    ax.plot(time, VA.T)
    ax.set_xlabel('Time (ms)')
    ax.set_ylabel('Membrane potential (mV)');
    plt.show()

QUESTION: see the influence of $I_0$ on the behavior, *when the noise amplitude is zero* :

In [ ]:
for rho in np.linspace(0.9, 1.1, 5):
    I_0_ = rho*150
    print('I_0_=', I_0_)
    VA = np.zeros((n_trials,len(time)))

    for i in range(n_trials):
        VA[i, :] = leaky_IF(time, genNoise(I_n=0, I_0=I_0_))

    print('number of spikes per trial :', (VA>0).sum(axis=1))
    fig, ax = plt.subplots(1, 1, figsize=(fig_width, fig_width/phi))
    ax.plot(time, VA.T)
    ax.set_xlabel('Time (ms)')
    ax.set_ylabel('Membrane potential (mV)');
    plt.show()

# Wrapping up

QUESTION: reproduce panel A: when the noise is zero, the traces of the neurons are not reproducible:

In [ ]:
seed = 2021
VA = np.zeros((n_trials,len(time)))
b_A = genNoise(I_n=0, I_0=30, seed=seed)

for i in range(n_trials):
    VA[i, :] = leaky_IF(time, b_A)


print('number of spikes per trial :', (VA>0).sum(axis=1))

fig, ax = plt.subplots(1, 1, figsize=(fig_width, fig_width/phi))
ax.plot(time, VA.T)
ax.set_xlabel('Time (ms)')
ax.set_ylabel('Membrane potential (mV)');

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(fig_width, fig_width/phi))
ax.eventplot([dt*np.where(VA.T[:, i] == Vspike)[0] for i in range(0, n_trials)], 
              colors='black', lineoffsets=1, linelengths=0.9);
ax.set_ylabel('Trial number')
ax.set_xlabel('Time (ms)')
ax.set_xlim(time.min(), time.max())
ax.set_ylim(-.5, n_trials-.5);

QUESTION: reproduce panel B: with frozen noise, the neuron traces are reproducible, even when the neuron has intrinsic noise:

In [ ]:
VB = np.zeros((n_trials, len(time)))
b_B = genNoise(I_n=300, I_0=15, seed=seed)
for i in range(n_trials):
    VB[i, :] = leaky_IF(time, b_B)

print('number of spikes per trial :', (VB>0).sum(axis=1))

fig, ax = plt.subplots(1, 1, figsize=(fig_width, fig_width/phi))
ax.plot(time, VB.T)
ax.set_xlabel('Time (ms)')
ax.set_ylabel('Membrane potential (mV)');

To sum up:

In [ ]:
fig, axs = plt.subplots(3, 2, figsize=(fig_width, fig_width))

axs[0][0].plot(time, b_A)
axs[0][1].plot(time, b_B)
axs[1][0].plot(time, VA.T)
axs[1][1].plot(time, VB.T)
axs[2][0].pcolor(time, range(n_trials), VA, vmax=Vthreshold, shading='nearest')
axs[2][1].pcolor(time, range(n_trials), VB, vmax=Vthreshold, shading='nearest')
for ax in axs.ravel(): 
    ax.set_xlabel('Time (ms)')
    ax.set_ylabel('Membrane potential (mV)');
axs[2][0].set_ylabel('trial #');
axs[2][1].set_ylabel('trial #');
for i in range(2):
    axs[0][i].set_ylabel('I_n (pA)')
    axs[0][i].set_ylim(0, 400);

QUESTION: conclude quickly: to what extent has the phenomenon been explained? What is the conclusion about the response of neurons to different dynamical signals?